# Pydantic AI with MCP and Ollama using Granite Models

**Author**: Vipul Mahajan(vipmaha1@in.ibm.com)

This notebook demonstrates how to integrate [Pydantic AI](https://ai.pydantic.dev), [Model Context Protocol (MCP)](https://modelcontextprotocol.io), and [Ollama](https://ollama.ai) with IBM Granite models to build agents that use external tools.

## Overview

### Pydantic AI
A Python agent framework for building production-grade AI applications with:
- Type-safe development with Pydantic models
- Structured, validated outputs
- Model-agnostic design

### Model Context Protocol (MCP)
An open standard for connecting AI applications to external tools and data:
- Standardized communication between models and services
- Dynamic tool discovery
- Reusable MCP servers across applications

### Ollama
Run LLMs locally on your machine:
- Complete privacy (data never leaves your machine)
- No API costs
- Offline capability

### What We'll Build
1. MCP servers exposing tools (calculator, text analyzer, statistics)
2. Pydantic AI agents that connect to MCP servers
3. Single-tool and multi-tool agentic workflows

## About this notebook

This notebook addresses a key challenge in building production-grade AI applications: how to enable language models to access external tools and data sources in a standardized, reusable way. 

Traditional approaches to tool calling often result in tightly-coupled code that's difficult to maintain and share across projects. The Model Context Protocol (MCP) solves this by providing an open standard for connecting AI applications to external services, while Pydantic AI ensures type-safe, validated interactions throughout your agent workflows.

In this notebook, you'll learn how to:
- Create MCP servers that expose tools (calculator, text analyzer, weather) using FastMCP
- Build Pydantic AI agents that connect to MCP servers and use their tools
- Implement both single-tool and multi-tool agentic workflows
- Run everything locally using Ollama with IBM Granite models for complete privacy and no API costs

## Prerequisites

Before running this notebook, ensure you have:

### 1. Ollama Installed and Running

Install Ollama from [ollama.ai](https://ollama.ai) and pull the Granite model:

```bash
# Install Ollama (macOS/Linux)
curl -fsSL https://ollama.ai/install.sh | sh

# Pull IBM Granite model
ollama pull granite3.3:8b

# Verify it's running
ollama list
```

### 2. Python Dependencies

We'll install the required packages in the next cell:

- `pydantic-ai-slim[mcp]` - Pydantic AI with MCP support
- `fastmcp` - For creating MCP servers
- `ollama` - Python Ollama client
- `httpx` - Async HTTP client for MCP
- Additional utilities

## Installation

Let's install all required dependencies:

In [ ]:
!echo "::group::Pip Install Dependencies"
%pip install "git+https://github.com/ibm-granite-community/utils.git" \
    "pydantic-ai-slim[openai]" \
    fastmcp \
    ollama \
    httpx \
    python-dotenv \
    mcp
!echo "::endgroup::"

## Setup and Configuration

Let's set up our imports and verify Ollama connectivity:

In [ ]:
import asyncio
import json
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import ollama
import subprocess
import sys
from ibm_granite_community.notebook_utils import wrap_text

print("🔍 Detecting available Ollama models...")
print("=" * 50)

try:
    # Get list of all models from Ollama
    models_response = ollama.list()
    
    # Extract model names
    all_models = []
    granite_models = []
    
    model_list = models_response.get('models', []) if hasattr(models_response, 'get') else getattr(models_response, 'models', [])
    
    for model_obj in model_list:
        
        model_name = getattr(model_obj, 'model', None) or (model_obj.get('model') if isinstance(model_obj, dict) else str(model_obj))
        
        if model_name:
            all_models.append(model_name)
            if 'granite' in model_name.lower():
                granite_models.append(model_name)
    
    # Select model
    if granite_models:
        print(f"✅ Ollama is running")
        print(f"✅ Found {len(granite_models)} Granite model(s):")
        for i, model in enumerate(granite_models):
            print(f"   [{i}] {model}")
        
        # 👇 CHANGE THIS INDEX TO SELECT A DIFFERENT MODEL
        GRANITE_MODEL = granite_models[0]  # 0 = first model, 1 = second, etc.
        
        print(f"\n🎯 Using: {GRANITE_MODEL}")
        if len(granite_models) > 1:
            print(f"   (To change: edit cell and modify granite_models[0])")
        
    elif all_models:
        print(f"⚠️  No Granite models found")
        print(f"✅ Found {len(all_models)} other model(s):")
        for i, model in enumerate(all_models):
            print(f"   [{i}] {model}")
        
        # 👇 CHANGE THIS INDEX TO SELECT A DIFFERENT MODEL
        GRANITE_MODEL = all_models[0]  # 0 = first model, 1 = second, etc.
        
        print(f"\n🎯 Using: {GRANITE_MODEL}")
        print(f"   💡 Install Granite: ollama pull granite3.3:8b")
        
    else:
        raise Exception("No models found in Ollama")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n🔧 QUICK FIX:")
    print("   1. Open terminal → Run: ollama serve")
    print("   2. Pull a model → Run: ollama pull granite3.3:8b")
    print("   3. Re-run this cell")
    
    # Check if models exist but server isn't running
    try:
        result = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0 and result.stdout:
            print("\n✅ Models installed (server not running):")
            print(result.stdout)
    except:
        pass
    
    GRANITE_MODEL = "granite3.3:latest"  # Fallback
    print(f"\n⚠️  Using fallback: {GRANITE_MODEL}")

print("=" * 50)

## Part 1: Creating an MCP Server

Let's start by creating a simple MCP server that exposes a few useful tools. We'll use FastMCP, which makes it easy to create MCP servers in Python.

Our server will provide three tools:
1. **Calculator** - Basic arithmetic operations
2. **Text Analyzer** - Count words, characters, and analyze sentiment
3. **Weather Info** - Simulated weather data (mock implementation)

In [ ]:
# Create a simple MCP server using FastMCP
# This will be saved as a separate Python file that runs as a subprocess

import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in dir() else os.getcwd()
if "Pydantic_AI_MCP" not in notebook_dir:
    # If we're not in the recipe directory, navigate to it
    notebook_dir = os.path.join(os.getcwd(), "recipes", "Pydantic_AI_MCP")

# Ensure the directory exists
os.makedirs(notebook_dir, exist_ok=True)

print(f"📁 Working directory: {notebook_dir}")

mcp_server_code = '''
from mcp.server.fastmcp import FastMCP
import math
from typing import Literal

# Initialize MCP server
mcp = FastMCP("Granite Tools Server")

@mcp.tool()
def calculator(operation: Literal["add", "subtract", "multiply", "divide", "power", "sqrt"], a: float, b: float = 0) -> str:
    """Perform basic mathematical calculations.
    
    Args:
        operation: The mathematical operation to perform
        a: First number
        b: Second number (not used for sqrt)
    
    Returns:
        The result of the calculation as a string
    """
    try:
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            if b == 0:
                return "Error: Division by zero"
            result = a / b
        elif operation == "power":
            result = a ** b
        elif operation == "sqrt":
            if a < 0:
                return "Error: Cannot take square root of negative number"
            result = math.sqrt(a)
        else:
            return f"Error: Unknown operation {operation}"
        
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@mcp.tool()
def text_analyzer(text: str) -> str:
    """Analyze text and return statistics.
    
    Args:
        text: The text to analyze
    
    Returns:
        A formatted string with text statistics
    """
    words = text.split()
    chars = len(text)
    chars_no_space = len(text.replace(" ", ""))
    sentences = text.count(".") + text.count("!") + text.count("?")
    
    # Simple sentiment analysis based on keywords
    positive_words = ["good", "great", "excellent", "amazing", "wonderful", "fantastic", "love", "best"]
    negative_words = ["bad", "terrible", "awful", "horrible", "worst", "hate", "poor"]
    
    text_lower = text.lower()
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        sentiment = "Positive"
    elif negative_count > positive_count:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    
    return f"""Text Analysis:
- Word count: {len(words)}
- Character count: {chars}
- Characters (no spaces): {chars_no_space}
- Sentence count: {sentences}
- Sentiment: {sentiment} (Positive: {positive_count}, Negative: {negative_count})"""

@mcp.tool()
def get_weather(city: str) -> str:
    """Get current weather information for a city (simulated).
    
    Args:
        city: The name of the city
    
    Returns:
        Weather information as a formatted string
    """
    # This is a mock implementation - in production, you'd call a real weather API
    import random
    
    conditions = ["Sunny", "Cloudy", "Rainy", "Partly Cloudy", "Windy"]
    condition = random.choice(conditions)
    temp = random.randint(50, 85)
    humidity = random.randint(30, 80)
    
    return f"""Weather for {city}:
- Condition: {condition}
- Temperature: {temp}°F
- Humidity: {humidity}%
- Note: This is simulated data for demonstration purposes"""

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

# Save the MCP server code to a file
server_path = os.path.join(notebook_dir, "mcp_server.py")
with open(server_path, "w") as f:
    f.write(mcp_server_code)

print(f"✅ MCP server code saved to: {server_path}")
print("\nServer provides the following tools:")
print("  1. calculator - Perform mathematical operations")
print("  2. text_analyzer - Analyze text statistics and sentiment")
print("  3. get_weather - Get weather information (simulated)")

## Part 2: Building a Pydantic AI Agent as MCP Client

Now let's create a Pydantic AI agent that connects to our MCP server and uses its tools. 

**What you'll learn in this section:**
- How to configure Pydantic AI with Ollama's OpenAI-compatible endpoint
- How to connect an agent to an MCP server
- How to implement a basic agentic loop with tool calling
- How to handle single-tool workflows

Pydantic AI doesn't have native Ollama support, but we can use Ollama's OpenAI-compatible API endpoint:

In [ ]:
# Configure Ollama to work with Pydantic AI
# Pydantic AI can use Ollama through OpenAI-compatible API
import httpx
from pydantic_ai.models import Model, KnownModelName
import os

# Set Ollama endpoint as OpenAI base URL
os.environ['OPENAI_BASE_URL'] = 'http://localhost:11434/v1'
os.environ['OPENAI_API_KEY'] = 'ollama'  # Dummy key for Ollama

# Use openai: prefix which will use the custom base URL
MODEL_NAME = f'openai:{GRANITE_MODEL}'

print(f"✅ Configured model: {MODEL_NAME}")
print(f"✅ Using Ollama endpoint: http://localhost:11434/v1")

In [ ]:
# Helper class to manage MCP server connection
class MCPServerManager:
    def __init__(self, server_script_path: str):
        self.server_script_path = server_script_path
        self.session = None
        self.read_stream = None
        self.write_stream = None
        self._client_context = None
    
    async def start(self):
        """Start the MCP server and initialize connection."""
        server_params = StdioServerParameters(
            command=sys.executable,  # Python executable
            args=[self.server_script_path],
        )
        
        self._client_context = stdio_client(server_params)
        self.read_stream, self.write_stream = await self._client_context.__aenter__()
        
        self.session = ClientSession(self.read_stream, self.write_stream)
        await self.session.__aenter__()
        await self.session.initialize()
        
        # List available tools
        tools_result = await self.session.list_tools()
        tool_names = [tool.name for tool in tools_result.tools]
        print(f"🔧 Connected to MCP server. Available tools: {tool_names}")
        
        return tools_result.tools
    
    async def call_tool(self, tool_name: str, arguments: dict) -> str:
        """Call a tool on the MCP server."""
        result = await self.session.call_tool(tool_name, arguments)
        
        # Extract result text
        if result.content:
            if isinstance(result.content, list):
                return "\n".join(
                    x.text if hasattr(x, "text") else str(x)
                    for x in result.content
                )
            return str(result.content)
        return "Tool executed successfully"
    
    async def close(self):
        """Clean up MCP server connection."""
        if self.session:
            await self.session.__aexit__(None, None, None)
        if self._client_context:
            await self._client_context.__aexit__(None, None, None)
        print("🔌 Disconnected from MCP server")

print("✅ MCPServerManager class defined")

In [ ]:
async def run_agent_with_mcp_tools(user_query: str, max_iterations: int = 3):
    """
    Simple agent that uses MCP tools to answer user queries.
    
    This is a minimal example showing the complete workflow:
    1. Connect to MCP server and discover available tools
    2. Create agent with system prompt describing tools
    3. Parse agent responses for tool calls
    4. Execute tools and feed results back to agent
    5. Return final answer to user
    
    💡 Copy this function as a starting point for your own MCP agents!
    
    Args:
        user_query: The question or task for the agent
        max_iterations: Maximum number of agent turns (prevents infinite loops)
    
    Returns:
        Final answer from the agent
    """
    
    print(f"🔍 Query: {user_query}")
    print(f"🔍 Using model: {GRANITE_MODEL}\n")
    
    # Step 1: Connect to MCP server
    server_path = os.path.join(notebook_dir, "mcp_server.py")
    mcp_manager = MCPServerManager(server_path)
    tools = await mcp_manager.start()
    
    # Step 2: Build tool descriptions for the agent
    tool_descriptions = []
    for tool in tools:
        # Extract parameter info from the tool schema
        params_desc = []
        if 'properties' in tool.inputSchema:
            for param, details in tool.inputSchema['properties'].items():
                param_type = details.get('type', 'any')
                param_desc = details.get('description', '')
                params_desc.append(f"  - {param} ({param_type}): {param_desc}")
        
        tool_desc = f"**{tool.name}**: {tool.description}\n" + "\n".join(params_desc)
        tool_descriptions.append(tool_desc)
    
    tools_info = "\n\n".join(tool_descriptions)
    
    # Step 3: Create agent with improved system prompt
    system_prompt = f"""You are a helpful assistant with access to tools.

Available tools:
{tools_info}

MANDATORY RESPONSE FORMATS - You MUST follow these EXACTLY:

FORMAT 1 - When calling a tool (use plain text, no markdown):
TOOL: <tool_name>
ARGS: <json_object>

FORMAT 2 - When providing final answer:
ANSWER: <your response>

CRITICAL RULES:
1. NEVER mix TOOL/ARGS with ANSWER in the same response
2. If a tool can help, respond with ONLY "TOOL:" and "ARGS:" lines (nothing else)
3. ARGS must be a valid JSON object with curly braces: {{"param": "value"}}
4. After you call a tool, you will receive its result in the next message
5. When you have the tool's result, respond with ONLY the "ANSWER:" line
6. Do NOT use markdown: write "TOOL:" not "**TOOL**:"
7. Do NOT use markdown: write "ANSWER:" not "**ANSWER**:"
8. Do NOT provide your own calculations or analysis - ALWAYS use the tool first

WORKFLOW EXAMPLE (calculator):

Turn 1 - User asks: "What is 5 to the power of 4?"
Your response:
TOOL: calculator
ARGS: {{"operation": "power", "a": 5, "b": 4}}

[System executes tool and returns: "Result: 625"]

Turn 2 - You receive tool result
Your response:
ANSWER: 5 raised to the power of 4 equals 625

WORKFLOW EXAMPLE (text_analyzer):

Turn 1 - User asks: "Analyze this text: 'Hello world'"
Your response:
TOOL: text_analyzer
ARGS: {{"text": "Hello world"}}

[System executes tool and returns analysis]

Turn 2 - You receive tool result  
Your response:
ANSWER: The text contains 2 words and 11 characters

KEY POINTS:
- In Turn 1: ONLY "TOOL:" and "ARGS:" (no answer, no explanation)
- In Turn 2: ONLY "ANSWER:" (no tool call, no formatting)
- Never guess the result - wait for the actual tool output"""

    agent = Agent(MODEL_NAME, system_prompt=system_prompt)
    
    # Step 4: Agent loop
    conversation_context = user_query
    
    for iteration in range(max_iterations):
        print(f"--- Iteration {iteration + 1} ---")
        
        # Get agent response
        result = await agent.run(conversation_context)
        
        # Extract response text (handle different result formats)
        if hasattr(result, 'data'):
            response = result.data
        elif hasattr(result, 'output'):
            response = result.output  
        elif hasattr(result, 'message'):
            response = result.message
        else:
            response = str(result)
        
        print(f"Agent: {response}\n")
        
        # Step 5: Clean up markdown formatting if present
        # Handle both **TOOL**: and **ARGS**: with potential spaces/newlines
        clean_response = response.replace("**TOOL:**", "TOOL:").replace("**TOOL**:", "TOOL:")
        clean_response = clean_response.replace("**ARGS:**", "ARGS:").replace("**ARGS**:", "ARGS:")
        clean_response = clean_response.replace("**ANSWER:**", "ANSWER:").replace("**ANSWER**:", "ANSWER:")
        
        # Check for mixed response (both TOOL and ANSWER in iteration 1)
        if "TOOL:" in clean_response and "ANSWER:" in clean_response:
            # Agent tried to provide answer before calling tool - extract only tool call
            print("⚠️  Agent included answer with tool call - extracting TOOL call only (will execute tool first)")
            clean_response = clean_response.split("ANSWER:")[0]
        
        # Parse response for tool call or answer
        if "TOOL:" in clean_response and "ARGS:" in clean_response:
            # Extract tool call
            lines = clean_response.strip().split('\n')
            tool_name = None
            args_json = None
            
            for line in lines:
                line = line.strip()
                if line.startswith("TOOL:"):
                    tool_name = line.replace("TOOL:", "").strip()
                elif line.startswith("ARGS:"):
                    args_json = line.replace("ARGS:", "").strip()
            
            if tool_name and args_json:
                try:
                    args = json.loads(args_json)
                    print(f"🔧 Calling tool: {tool_name}")
                    print(f"📋 Arguments: {json.dumps(args, indent=2)}")
                    
                    # Execute tool via MCP
                    tool_result = await mcp_manager.call_tool(tool_name, args)
                    print(f"✅ Tool Result: {tool_result}\n")
                    
                    # Clearer follow-up instruction with strict format requirement
                    conversation_context = f"""The user asked: {user_query}

You called the {tool_name} tool and received this result:
{tool_result}

Now respond with ONLY this format (nothing else):
ANSWER: <your answer based on the tool result>

Do NOT include any other text, formatting, or explanations. Just the single ANSWER: line."""
                    
                except json.JSONDecodeError as e:
                    print(f"❌ Failed to parse tool arguments: {e}")
                    print(f"   Raw args_json: {args_json}")
                    break
                except Exception as e:
                    print(f"❌ Tool execution failed: {e}")
                    break
            else:
                print(f"❌ Could not extract tool name or arguments")
                print(f"   Tool: {tool_name}, Args: {args_json}")
                break
                
        elif "ANSWER:" in clean_response:
            # Final answer received
            answer = clean_response.split("ANSWER:", 1)[1].strip()
            print(f"{'='*50}")
            print(f"✅ Final Answer: {answer}")
            print(f"{'='*50}")
            await mcp_manager.close()
            return answer
            
        else:
            # Response doesn't match expected format
            print(f"⚠️  Agent response doesn't match expected format")
            print(f"   Expected: 'TOOL: <name>' + 'ARGS: <json>' OR 'ANSWER: <text>'")
            print(f"   Got: {clean_response[:200]}...")
            
            # Try to extract answer if it's prose without "ANSWER:" prefix
            if iteration > 0:  # Only in iteration 2+
                print(f"ℹ️  Attempting to extract answer from prose response...")
                # Treat the whole response as the answer
                print(f"{'='*50}")
                print(f"✅ Final Answer: {clean_response.strip()}")
                print(f"{'='*50}")
                await mcp_manager.close()
                return clean_response.strip()
            else:
                break
    
    # Max iterations reached
    print(f"\n⚠️  Reached maximum iterations ({max_iterations})")
    await mcp_manager.close()
    return "Agent did not provide a final answer within iteration limit."

print("✅ run_agent_with_mcp_tools() function defined")
print("💡 This function handles single or multi-tool workflows")
print("💡 Copy this code to your projects and customize as needed!")

### Agent with MCP Tools

Now let's create an agent that can use our MCP server's tools. We'll manually handle tool calling since Pydantic AI's native MCP client support is still evolving:

In [ ]:
# Test 1: Calculator tool
await run_agent_with_mcp_tools("What is 9 raised to the power of 3?")

In [ ]:
# Test 2: Text analyzer tool
await run_agent_with_mcp_tools(
    "Analyze this text: 'IBM Granite models are fantastic! They provide excellent performance and amazing capabilities.'"
)

In [ ]:
# Test 3: Weather tool
await run_agent_with_mcp_tools("What's the weather like in San Francisco?")

## Part 3: Advanced Multi-Tool Agent

Now let's build an advanced agent that can coordinate multiple tools to accomplish complex tasks. This demonstrates a production-ready pattern where the agent:

1. Analyzes the user's request
2. Breaks it into sub-tasks (e.g., data analysis + formatting)
3. Calls multiple tools sequentially
4. Synthesizes the final answer

### Example Workflow:
```
User: "Analyze these sales numbers: 120, 450, 380, 290, 510 and format as markdown"

Step 1: Agent calls data_statistics → Gets mean, median, etc.
Step 2: Agent calls format_data → Formats results as markdown
Step 3: Agent provides formatted business report
```

Let's create the advanced MCP server with data analysis and formatting tools:

In [ ]:
# Create advanced MCP server with data analysis and formatting tools
advanced_mcp_server_code = '''
from mcp.server.fastmcp import FastMCP
import statistics
from typing import Literal

# Initialize MCP server
mcp = FastMCP("Advanced Analytics Server")

@mcp.tool()
def data_statistics(numbers: str) -> str:
    """Calculate statistical measures for a list of numbers.
    
    Args:
        numbers: Comma-separated list of numbers (e.g., "10, 20, 30, 40")
    
    Returns:
        Statistical analysis including mean, median, mode, stdev
    """
    try:
        # Parse numbers
        num_list = [float(x.strip()) for x in numbers.split(',')]
        
        if len(num_list) == 0:
            return "Error: No numbers provided"
        
        # Calculate statistics
        mean = statistics.mean(num_list)
        median = statistics.median(num_list)
        
        # Mode (handle potential StatisticsError)
        try:
            mode = statistics.mode(num_list)
        except statistics.StatisticsError:
            mode = "No unique mode"
        
        # Standard deviation (requires at least 2 values)
        if len(num_list) >= 2:
            stdev = statistics.stdev(num_list)
        else:
            stdev = "N/A (need 2+ values)"
        
        # Min and max
        minimum = min(num_list)
        maximum = max(num_list)
        
        return f"""Statistics for {len(num_list)} numbers:
- Mean (average): {mean}
- Median: {median}
- Mode: {mode}
- Standard Deviation: {stdev}
- Minimum: {minimum}
- Maximum: {maximum}
- Range: {maximum - minimum}"""
        
    except ValueError as e:
        return f"Error parsing numbers: {e}"
    except Exception as e:
        return f"Error: {e}"

@mcp.tool()
def format_data(data: str, format_type: Literal["markdown", "json", "csv", "plain"]) -> str:
    """Format data in different output formats.
    
    Args:
        data: The data to format (text with key-value pairs or statistics)
        format_type: Output format (markdown, json, csv, or plain)
    
    Returns:
        Formatted data string
    """
    try:
        if format_type == "markdown":
            # Convert to markdown table or formatted text
            lines = data.strip().split('\\n')
            result = "## Data Analysis Results\\n\\n"
            
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    result += f"**{key.strip()}:** {value.strip()}\\n"
                else:
                    result += f"{line}\\n"
            
            return result
            
        elif format_type == "json":
            # Convert to JSON format
            import json
            lines = data.strip().split('\\n')
            result_dict = {}
            
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    # Try to convert to number
                    try:
                        result_dict[key.strip()] = float(value.strip())
                    except:
                        result_dict[key.strip()] = value.strip()
            
            return json.dumps(result_dict, indent=2)
            
        elif format_type == "csv":
            # Convert to CSV format
            lines = data.strip().split('\\n')
            result = "Metric,Value\\n"
            
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    result += f"{key.strip()},{value.strip()}\\n"
            
            return result
            
        else:  # plain
            return data
            
    except Exception as e:
        return f"Error formatting data: {e}"

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

# Save the advanced MCP server
advanced_server_path = os.path.join(notebook_dir, "advanced_mcp_server.py")
with open(advanced_server_path, "w") as f:
    f.write(advanced_mcp_server_code)

print(f"✅ Advanced MCP server created: {advanced_server_path}")
print("\\nServer provides the following tools:")
print("  1. data_statistics - Calculate mean, median, mode, stdev, etc.")
print("  2. format_data - Format data as markdown, JSON, CSV, or plain text")

In [ ]:
# Test the advanced agent with a complex query requiring multiple tools
complex_query = """I have these sales numbers: 120, 450, 380, 290, 510. 
Please analyze them and format the results as markdown."""

async def run_advanced_agent(user_query: str, max_iterations: int = 5):
    """
    Advanced agent that coordinates multiple tools to accomplish complex tasks.
    
    This demonstrates a production-ready pattern:
    1. Agent analyzes the query
    2. Breaks it into sub-tasks (data analysis, formatting)
    3. Calls multiple tools in sequence
    4. Synthesizes final answer
    
    💡 This pattern works for: research agents, data pipelines, multi-step workflows
    
    Args:
        user_query: Complex query requiring multiple tool calls
        max_iterations: Maximum loop iterations (prevents runaway agents)
    
    Returns:
        Final answer from the agent
    """
    
    print(f"🔍 Using model: {GRANITE_MODEL}")
    print(f"🔍 Model name configured: {MODEL_NAME}\n")
    
    # Step 1: Connect to MCP server
    advanced_server_path = os.path.join(notebook_dir, "advanced_mcp_server.py")
    mcp_manager = MCPServerManager(advanced_server_path)
    tools = await mcp_manager.start()
    
    # Step 2: Build tool descriptions
    tool_descriptions = []
    for tool in tools:
        params_desc = []
        if 'properties' in tool.inputSchema:
            for param, details in tool.inputSchema['properties'].items():
                param_type = details.get('type', 'any')
                param_desc = details.get('description', '')
                params_desc.append(f"  - {param} ({param_type}): {param_desc}")
        
        tool_desc = f"**{tool.name}**: {tool.description}\n" + "\n".join(params_desc)
        tool_descriptions.append(tool_desc)
    
    tools_info = "\n\n".join(tool_descriptions)
    
    # Step 3: Create system prompt
    system_prompt = f"""You are a STRICT tool-calling assistant. Follow these rules EXACTLY.

Available tools:
{tools_info}

MANDATORY WORKFLOW FOR THIS QUERY:
The user wants data analyzed AND formatted. You MUST:
1. First call data_statistics to get the actual statistics
2. Then call format_data to format those statistics  
3. Finally provide ANSWER with the formatted result

CRITICAL RULES:
1. Call ONLY ONE tool per response
2. WAIT for the tool result before proceeding
3. Do NOT call multiple tools in one response
4. Do NOT provide ANSWER until BOTH tools have been called
5. Do NOT guess or calculate - use the actual tool results

RESPONSE FORMAT:
TOOL: <exactly one tool name>
ARGS: {{"param": "value"}}

ABSOLUTELY FORBIDDEN:
❌ Calling 2+ tools in one response
❌ Including ANSWER before all tools are called
❌ Guessing results without calling tools
❌ Providing answers based on predicted results

WORKFLOW EXAMPLE:
Turn 1: 
TOOL: data_statistics
ARGS: {{"numbers": "..."}}

Turn 2 (after seeing result):
TOOL: format_data
ARGS: {{"data": "<actual result from step 1>", "format_type": "markdown"}}

Turn 3 (after seeing formatted result):
ANSWER: <the formatted markdown result>

Remember: ONE tool per turn. WAIT for results. NO ANSWER until BOTH tools called."""

    agent = Agent(MODEL_NAME, system_prompt=system_prompt)
    
    print(f"\n{'='*60}")
    print(f"User Query: {user_query}")
    print(f"{'='*60}\n")
    
    conversation_context = user_query
    tool_results = []
    tools_called = set()  # Track which tools have been called
    
    # Step 4: Agent loop with strict enforcement
    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1} ---")
        
        # Get agent response
        result = await agent.run(conversation_context)
        
        # Extract response text
        if hasattr(result, 'data'):
            response = result.data
        elif hasattr(result, 'output'):
            response = result.output
        elif hasattr(result, 'message'):
            response = result.message
        else:
            response = str(result)
        
        print(f"Agent response:\n{response}\n")
        
        # Step 5: Strict enforcement - block premature ANSWER
        if "ANSWER:" in response:
            # Check if both required tools were called
            required_tools = {'data_statistics', 'format_data'}
            if not required_tools.issubset(tools_called):
                missing = required_tools - tools_called
                print(f"⚠️  Agent tried to provide ANSWER too early!")
                print(f"   Missing tool calls: {missing}")
                print(f"   Forcing agent to call missing tools first...\n")
                
                # Force agent to call the missing tool
                next_tool = list(missing)[0]
                if next_tool == 'data_statistics':
                    conversation_context = f"""You MUST call data_statistics first to get the actual statistics.

Do NOT provide an answer yet. Respond with ONLY:
TOOL: data_statistics
ARGS: {{"numbers": "120, 450, 380, 290, 510"}}"""
                else:
                    conversation_context = f"""You have the statistics. Now you MUST call format_data to format them.

Use the ACTUAL result from data_statistics:
{tool_results[-1]['result'] if tool_results else ''}

Respond with ONLY:
TOOL: format_data
ARGS: {{"data": "<actual statistics>", "format_type": "markdown"}}"""
                continue
            
            # Both tools called - extract answer
            answer_part = response.split("ANSWER:", 1)[1]
            if "TOOL:" in answer_part:
                answer_part = answer_part.split("TOOL:")[0]
            
            answer = answer_part.strip()
            print(f"\n{'='*60}")
            print(f"✅ FINAL ANSWER:")
            print(f"{'='*60}")
            print(answer)
            break
        
        # Step 6: Parse tool call
        elif "TOOL:" in response and "ARGS:" in response:
            tool_name = None
            args_json = None
            
            lines = response.split('\n')
            tool_line_idx = None
            
            # Find first TOOL: line
            for idx, line in enumerate(lines):
                line = line.strip()
                if line.startswith("TOOL:"):
                    tool_name = line.replace("TOOL:", "").strip()
                    tool_line_idx = idx
                    break
            
            # Find ARGS: after the TOOL: line
            if tool_line_idx is not None:
                remaining_lines = lines[tool_line_idx:]
                response_segment = '\n'.join(remaining_lines)
                
                # Look for next TOOL: to limit search
                next_tool_idx = response_segment.find('\nTOOL:', 1)
                if next_tool_idx > 0:
                    response_segment = response_segment[:next_tool_idx]
                
                # Find ARGS in this segment
                if "ARGS:" in response_segment:
                    args_start = response_segment.find("ARGS:")
                    after_args = response_segment[args_start + 5:].strip()
                    
                    # Extract JSON (handle multi-line)
                    if after_args.startswith('{'):
                        brace_count = 0
                        json_end = 0
                        for i, char in enumerate(after_args):
                            if char == '{':
                                brace_count += 1
                            elif char == '}':
                                brace_count -= 1
                                if brace_count == 0:
                                    json_end = i + 1
                                    break
                        
                        if json_end > 0:
                            args_json = after_args[:json_end]
            
            if tool_name and args_json:
                # Check if agent tried multiple tools
                tool_count = response.count('TOOL:')
                if tool_count > 1:
                    print(f"ℹ️  Agent mentioned {tool_count} tools, executing first: {tool_name}\n")
                
                try:
                    args = json.loads(args_json)
                    print(f"🔧 Calling tool: {tool_name}")
                    print(f"📋 Arguments: {json.dumps(args, indent=2)}")
                    
                    # Execute tool
                    tool_result = await mcp_manager.call_tool(tool_name, args)
                    print(f"\n✅ Tool Result:\n{tool_result}\n")
                    
                    tool_results.append({
                        "tool": tool_name,
                        "result": tool_result
                    })
                    tools_called.add(tool_name)
                    
                    # Build follow-up with specific instructions
                    if 'data_statistics' in tools_called and 'format_data' not in tools_called:
                        # Just called data_statistics - MUST call format_data next
                        conversation_context = f"""You called data_statistics and got this result:
{tool_result}

Now you MUST call format_data to format these statistics as markdown.

Respond with ONLY (use the ACTUAL result above):
TOOL: format_data
ARGS: {{"data": "<paste the statistics result here>", "format_type": "markdown"}}

Do NOT provide ANSWER yet. Call format_data first."""
                    
                    elif 'format_data' in tools_called:
                        # Just called format_data - now can provide answer
                        conversation_context = f"""You have called both required tools:

1. data_statistics result:
{tool_results[0]['result']}

2. format_data result:
{tool_results[1]['result']}

Now provide the final answer using ONLY:
ANSWER: <the formatted markdown result>"""
                    
                except json.JSONDecodeError as e:
                    print(f"❌ Failed to parse JSON: {e}")
                    break
                except Exception as e:
                    print(f"❌ Tool execution error: {e}")
                    break
            else:
                print(f"❌ Could not extract tool name or arguments")
                break
        
        else:
            print(f"⚠️  Invalid response format")
            break
    
    # Cleanup
    await mcp_manager.close()
    
    # Summary
    print(f"\n\n📊 Execution Summary:")
    print(f"   - Model: {GRANITE_MODEL}")
    print(f"   - Iterations: {iteration + 1}/{max_iterations}")
    print(f"   - Tools called: {len(tool_results)}")
    for i, r in enumerate(tool_results, 1):
        print(f"     {i}. {r['tool']}")

# Test the advanced agent
print("\n🎯 Testing the advanced agent:\n")
await run_advanced_agent(complex_query)

## Conclusion

In this notebook, you learned how to build production-grade AI agents by combining three powerful technologies: Pydantic AI for type-safe agent development, the Model Context Protocol (MCP) for standardized tool integration, and Ollama for local model deployment with IBM Granite models.

You explored two key patterns:
1. **Single-tool workflows** - Agents that call one tool at a time (calculator, text analyzer, weather)
2. **Multi-tool workflows** - Advanced agents that coordinate multiple tools sequentially (data analysis + formatting)

These patterns form the foundation for building complex agentic systems that can research information, process data pipelines, and execute multi-step workflows.

### Next Steps

Now that you understand the basics, you can:
- **Build custom MCP servers** - Add your own tools (database queries, API calls, file operations)
- **Create specialized agents** - Develop agents for specific domains (research, data analysis, customer service)
- **Scale to multi-agent systems** - Coordinate multiple agents working together on complex tasks
- **Integrate with watsonx.ai** - Connect to IBM's cloud platform for enterprise-grade deployments

### Related Resources

Explore more AI agent patterns in the Granite Cookbook:
- [AI Agents Recipes](https://github.com/ibm-granite-community/granite-snack-cookbook/tree/main/recipes/AI-Agents) - ReAct agents, Agentic RAG, and specialized agents
- [Structured Response Recipe](https://github.com/ibm-granite-community/granite-snack-cookbook/tree/main/recipes/Structured_Response) - Type-safe outputs with Pydantic
- [Function Calling Recipe](https://github.com/ibm-granite-community/granite-snack-cookbook/tree/main/recipes/Function-Calling) - Native function calling with Granite models

## References

### Official Documentation
- [Pydantic AI Documentation](https://ai.pydantic.dev) - Complete guide to Pydantic AI framework
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io) - Official MCP specification and documentation
- [Ollama](https://ollama.ai) - Run large language models locally
- [FastMCP](https://github.com/jlowin/fastmcp) - Python framework for building MCP servers
- [IBM Granite Models](https://www.ibm.com/granite) - IBM's family of open-source foundation models

### Related Tutorials
- [Pydantic AI Getting Started](https://ai.pydantic.dev/getting-started/) - Quick start guide for Pydantic AI
- [MCP Servers Repository](https://github.com/modelcontextprotocol/servers) - Collection of example MCP servers
- [Ollama Model Library](https://ollama.ai/library) - Browse available models for Ollama